In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [5]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [6]:


driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 
x = 0
for x in range(142):
    button.click()    
    time.sleep(1)
    x+=1
    print(x)
page_source = driver.page_source # save the source to feed to beautiful soup 
print("saved")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
saved


In [7]:
soup = BeautifulSoup(page_source)


In [8]:
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [9]:
#url_soup

In [10]:
url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [17]:
url_list  = [*set(url_list)] 

In [18]:
len(url_list)

813

In [34]:
# verifying for one instance
url = url_list[100]
#url = 'https://www.foxnews.com/opinion/tucker-carlson-faa-travel-pete-buttigieg'
response = requests.get(url)
soup = BeautifulSoup(response.content)
title = soup.title.string

#results = soup.find(id='wrapper')
#body = soup.find_all('p')
#text = body[1].text

a_tag = soup.find_all('a')
for tag in a_tag:
    tag.decompose()

copy_tag = soup.find_all('p', class_ = "copyright")
for tag in copy_tag:
    tag.decompose()
    
span_tag = soup.find_all('span')
for tag in span_tag:
    tag.decompose()
    
bold_tag = soup.find_all('strong') 
for tag in bold_tag:
    tag.decompose()
    
italic_tag=soup.find_all('i') 
for tag in italic_tag:
    tag.decompose()

other_tag = soup.find_all('p', class_ ="dek")
for tag in other_tag:
    tag.decompose()
    
other_tag2 = soup.find_all('p', class_ ="subscribed hide")
for tag in other_tag2:
    tag.decompose()
    
other_tag3 = soup.find_all('p', class_ ="success hide") 
for tag in other_tag3:
    tag.decompose()



final = soup.find_all('p')
# text=''
# print(final)
finaltext = ""
for p in final:

    try:
        if len(p.text) > 1:
            finaltext = finaltext + p.text + '\n'+ '\n'
            finaltext = finaltext.replace('\xa0', ' ')

    except:
        continue
print(finaltext)
# #finaltext = ' '.join(text)
# #text
# # #soup.find_all('p')
# #soup.find('time').text

Fox News host gives his take on pro-abortion protesters targeting Supreme Court justices over the possible overturn of Roe v. Wade on 'Tucker Carlson Tonight.'

It's pretty hard to argue with people who are passive-aggressive. You may have tried it before. "Why are you so angry?" they scream. "Stop being violent," they snarl as they punch you in the face. Passive-aggressive people are intent on dominating you, but they're too dishonest to admit it.  

Now, it's not an honorable style of attack, but it's very effective, mostly because it's so bewildering. The Democratic Party practices this. Democrats will never meet you on an open field of battle. Instead, they will sneak up behind you and knock you unconscious with a bag of sanctimony. This is the party of weak men and angry women, so passive aggression is their only mode of communication. If you've ever seen one of Jen Psaki's press conferences, you know exactly what we're talking about. 

We watched one of them yesterday, in fact. I

In [36]:

len(url_list)

813

In [37]:

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:
        try:
            if len(p.text) > 1:
                finaltext = finaltext + p.text + '\n'+ '\n'
                finaltext = finaltext.replace('\xa0', ' ')

        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'text': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)

full_df = pd.concat(dfs).reset_index(drop=True)

In [41]:
full_df

,url,timestamp,title,text
0,https://www.foxnews.com/opinion/tucker-carlson...,"October 7, 2021 11:37pm EDT",Tucker Carlson: It is beginning to seem like t...,'Tucker Carlson Tonight' host sounds off on Bi...
1,https://www.foxnews.com/opinion/tucker-reveals...,"April 15, 2022 11:08pm EDT",Tucker reveals compelling new Fox Nation seaso...,Senior executive producer Justin Wells discuss...
2,https://www.foxnews.com/opinion/tucker-carlson...,"June 25, 2019 5:57am EDT",Tucker Carlson: Trump attacked by both sides f...,President Trump met with criticism after calli...
3,https://www.foxnews.com/opinion/tucker-carlson...,"August 10, 2021 12:05am EDT",Tucker Carlson: You can be sophisticated or a ...,'Tucker Carlson Tonight' host discusses differ...
4,https://www.foxnews.com/opinion/tucker-carlson...,"October 16, 2020 10:23pm EDT",Tucker Carlson: Biden ABC News town hall was a...,'Tucker Carlson Tonight' host compares and con...
...,...,...,...,...
808,https://www.foxnews.com/opinion/tucker-carlson...,"December 7, 2020 10:07pm EST",Tucker Carlson: Our elites' collusion with Chi...,'Tucker Carlson Tonight' host calls out powerf...
809,https://www.foxnews.com/opinion/tucker-carlson...,"September 14, 2021 11:22pm EDT","Tucker Carlson: Mark Milley committed treason,...",Fox News host reacts to Mark Milley's comments...
810,https://www.foxnews.com/opinion/tucker-carlson...,"January 3, 2020 10:00am EST",Tucker Carlson: Normal people don't want crimi...,Pre-trial detention and cash bail have been el...
811,https://www.foxnews.com/opinion/tucker-carlson...,"October 18, 2022 10:47pm EDT",TUCKER CARLSON: Elon Musk is committing this u...,Fox News host Tucker Carlson explains why free...


In [42]:
print(full_df.iloc[19,3])

Fox News host shreds Treasury Secretary Janet Yellen as Americans experience record-high inflation on 'Tucker Carlson Tonight.'

It was six years ago and one of those semi-hilarious moments that has unfortunately been lost to history that Janet Yellen sat through what must have been the single most uncomfortable congressional hearing of her life. At the time, you may remember, Yellen was running the Federal Reserve and she was, as she remains today, a lifestyle liberal in good standing.  

So, put yourself in her position and imagine the shock she must have felt when members of the Congressional Black Caucus suggested that she, Janet Yellen, was, in fact, a racist, maybe even a White supremacist. What did Janet Yellen do wrong? Well, they suggested what she did wrong was she failed to harness the power of the Federal Reserve to help African-Americans specifically. Now, at first, Yellen seemed confused by this complaint. The point of the Fed is to keep the American economy stable. The p

In [43]:
#no_duplicates=full_df.drop_duplicates().reset_index(drop=True)

In [49]:
full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])

C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [53]:
full_df=full_df.sort_values(by=['timestamp']).reset_index(drop=True)

In [1]:
full_df

NameError: name 'full_df' is not defined

In [56]:
full_df.to_csv('Tucker_transcripts_2.17.23.tsv', sep="\t")